# 임베디드 시스템 경진대회 코드

## Servo X

In [ ]:
!pip install pyserial
!pip install cvzone
!pip install opencv-python
!pip install mediapipe --user

In [8]:
import cv2
import numpy as np
import time
import HandTrackingModule as htm
import math
import serial

ser = serial.Serial('COM8', 9600)

def send_right(right,state):
    ser.write(f"R{right},{state}\n".encode())
    print(f"Sent right: {right}, state: {state}")

def send_left(left, state):
    ser.write(f"L{left}, {state}\n".encode())
    print(f"Sent left: {left}, state: {state}")
    
def check_hand_state(fingers):
    # 엄지만 올라가 있고 나머지는 접혀있는 상태
    if fingers[0] == 1 and sum(fingers[1:]) == 0:
        return "Back"
    # 주먹을 쥔 상태 (모든 손가락이 접혀있음)
    elif sum(fingers) == 0:
        return "Fist"
    # 소지만 올라가 있고 나머지는 접혀있는 상태
    elif fingers[0] == 0 and fingers[1] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[4] == 1:
        return "Front"
    else:
        return "Other"

def display_hand_state(img, left_state, right_state, left_hand, right_hand):
    # 왼손 상태 표시
    if left_state == "Back":
        cv2.putText(img, "LeftBack", (10, hCam - 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    else:
        cv2.putText(img, "LeftFront", (10, hCam - 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    # 오른손 상태 표시
    if right_state == "Back":
        cv2.putText(img, "RightBack", (wCam - 150, hCam - 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(img, "RightFront", (wCam - 150, hCam - 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # 주먹 상태에 따른 텍스트 표시
    if len(left_hand) != 0 and check_hand_state(detector.fingersUpDown(left_hand, "Left")) == "Fist":
        cv2.putText(img, "Left Fist", (10, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    if len(right_hand) != 0 and check_hand_state(detector.fingersUpDown(right_hand, "Right")) == "Fist":
        cv2.putText(img, "Right Fist", (wCam - 150, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

left_state = "Front"
right_state = "Front"

wCam, hCam = 640, 480

cap = cv2.VideoCapture(1)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

detector = htm.handDetector(detectionCon=0.7)

while True:
    success, img = cap.read()
    img = detector.findHands(img)
    left_hand, right_hand = detector.findPosition(img, draw=False)

    # 왼손 처리
    if left_hand:
        # 엄지와 검지 랜드마크 (예: 4번과 8번)
        x1, y1 = left_hand[4][1], left_hand[4][2]
        x2, y2 = left_hand[8][1], left_hand[8][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2
        
        cv2.circle(img, (x1, y1), 5, (255, 0, 0), cv2.FILLED)  # 파란색
        cv2.circle(img, (x2, y2), 5, (255, 0, 0), cv2.FILLED)  # 파란색
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 2)  # 파란색 선
        
        cv2.circle(img, (cx,cy), 5, (255,0,255), cv2.FILLED)
        
        left_length = int(math.hypot(x2-x1, y2-y1))
        
        if left_length<=15:
            cv2.circle(img, (cx,cy), 8, (0,255,0), cv2.FILLED)
            send_left(0, "F")
            time.sleep(0.1)
        
        # 주먹여부처리
        fingers_left = detector.fingersUpDown(left_hand, "Left")
        current_left_state = check_hand_state(fingers_left)
        
        # 상태 유지 로직
        if current_left_state == "Back":
            left_state = "Back"
        elif current_left_state == "Front":
            left_state = "Front"
        elif left_state == "Back" and current_left_state != "Front":
            # Back 상태를 유지
            pass
        else:
            left_state = current_left_state
        
        #아두이노 값 전송
        if left_state == "Back":
            send_left(left_length, "B")
            time.sleep(0.1)
        else:
            send_left(left_length, "F")
            time.sleep(0.1)
        

    # 오른손 처리
    if right_hand:
        # 엄지와 검지 랜드마크 (예: 4번과 8번)
        x1, y1 = right_hand[4][1], right_hand[4][2]
        x2, y2 = right_hand[8][1], right_hand[8][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2
        
        cv2.circle(img, (x1, y1), 5, (0, 0, 255), cv2.FILLED)  # 빨간색
        cv2.circle(img, (x2, y2), 5, (0, 0, 255), cv2.FILLED)  # 빨간색
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)  # 빨간색 선
        
        cv2.circle(img, (cx,cy), 5, (255,0,255), cv2.FILLED)
        
        right_length = int(math.hypot(x2-x1, y2-y1))
        
        if right_length<=15:
            cv2.circle(img, (cx,cy), 8, (0,255,0), cv2.FILLED)
            send_right(0, "F")
            time.sleep(0.1)
        
         #주먹여부처리
        fingers_right = detector.fingersUpDown(right_hand, "Right")
        current_right_state = check_hand_state(fingers_right)
        
        # 상태 유지 로직
        if current_right_state == "Back":
            right_state = "Back"
        elif current_right_state == "Front":
            right_state = "Front"
        elif right_state == "Back" and current_right_state != "Front":
            # Back 상태를 유지
            pass
        else:
            right_state = current_right_state
        
        if right_state == "Back":
            send_right(right_length, "B")
            time.sleep(0.1)
        else:
            send_right(right_length, "F")
            time.sleep(0.1)

    # 여기에 display_hand_state 함수 호출
    display_hand_state(img, left_state, right_state, left_hand, right_hand)
    
    # fps 확인
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    
    cv2.putText(img, f'FPS: {int(fps)}', (40, 80), cv2.FONT_HERSHEY_COMPLEX,
                1, (255, 0, 0), 3)
    
    cv2.imshow("Img", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키를 누르면 종료
        break
        
ser.close()
cap.release()
cv2.destroyAllWindows()

Sent right: 0, state: F
Sent right: 8, state: F
Sent left: 81, state: F
Sent left: 75, state: F
Sent left: 51, state: B
Sent left: 74, state: B
Sent right: 0, state: F
Sent right: 13, state: F
Sent left: 38, state: B
Sent left: 81, state: B
Sent left: 36, state: B
Sent right: 79, state: F
Sent left: 93, state: B
Sent right: 120, state: F
Sent right: 107, state: F
Sent left: 123, state: B
Sent right: 161, state: F
Sent right: 34, state: F
Sent right: 0, state: F
Sent right: 6, state: F
Sent left: 0, state: F
Sent left: 6, state: B
Sent right: 48, state: F
Sent left: 16, state: B
Sent left: 0, state: F
Sent left: 10, state: B
Sent right: 0, state: F
Sent right: 12, state: F
Sent left: 0, state: F
Sent left: 14, state: B
Sent right: 0, state: F
Sent right: 9, state: F
Sent left: 0, state: F
Sent left: 8, state: B
Sent right: 0, state: F
Sent right: 10, state: F
Sent left: 0, state: F
Sent left: 7, state: B
Sent right: 0, state: F
Sent right: 9, state: F
Sent left: 0, state: F
Sent left: 7

## Servo O

In [ ]:
import cv2
import numpy as np
import time
import HandTrackingModule as htm
import math
import serial

ser = serial.Serial('COM5', 9600)

def send_right(right,state):
    ser.write(f"R{right},{state}\n".encode())
    print(f"Sent right: {right}, state: {state}")

def send_left(left, state):
    ser.write(f"L{left}, {state}\n".encode())
    print(f"Sent left: {left}, state: {state}")
    
def send_servo(state):
    ser.write(f"S{state}\n".encode())
    print(f"Sent servo state: {state}")
    
def check_hand_state(fingers):
    # 엄지만 올라가 있고 나머지는 접혀있는 상태
    if fingers[0] == 1 and sum(fingers[1:]) == 0:
        return "Back"
    # 주먹을 쥔 상태 (모든 손가락이 접혀있음)
    elif sum(fingers) == 0:
        return "Fist"
    # 소지만 올라가 있고 나머지는 접혀있는 상태
    elif fingers[0] == 0 and fingers[1] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[4] == 1:
        return "Front"
    else:
        return "Other"

def display_hand_state(img, left_state, right_state, left_hand, right_hand):
    # 왼손 상태 표시
    if left_state == "Back":
        cv2.putText(img, "LeftBack", (10, hCam - 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    else:
        cv2.putText(img, "LeftFront", (10, hCam - 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    # 오른손 상태 표시
    if right_state == "Back":
        cv2.putText(img, "RightBack", (wCam - 150, hCam - 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(img, "RightFront", (wCam - 150, hCam - 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # 주먹 상태에 따른 텍스트 표시
    if len(left_hand) != 0 and check_hand_state(detector.fingersUpDown(left_hand, "Left")) == "Fist":
        cv2.putText(img, "Left Fist", (10, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    if len(right_hand) != 0 and check_hand_state(detector.fingersUpDown(right_hand, "Right")) == "Fist":
        cv2.putText(img, "Right Fist", (wCam - 150, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

left_state = "Front"
right_state = "Front"

wCam, hCam = 640, 480

cap = cv2.VideoCapture(1)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

detector = htm.handDetector(detectionCon=0.7)

while True:
    success, img = cap.read()
    img = detector.findHands(img)
    left_hand, right_hand = detector.findPosition(img, draw=False)

    # 왼손 처리
    if left_hand:
        # 엄지와 검지 랜드마크 (예: 4번과 8번)
        x1, y1 = left_hand[4][1], left_hand[4][2]
        x2, y2 = left_hand[8][1], left_hand[8][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2
        
        cv2.circle(img, (x1, y1), 5, (255, 0, 0), cv2.FILLED)  # 파란색
        cv2.circle(img, (x2, y2), 5, (255, 0, 0), cv2.FILLED)  # 파란색
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 2)  # 파란색 선
        
        cv2.circle(img, (cx,cy), 5, (255,0,255), cv2.FILLED)
        
        left_length = int(math.hypot(x2-x1, y2-y1))
        
        if left_length<=15:
            cv2.circle(img, (cx,cy), 8, (0,255,0), cv2.FILLED)
            send_left(0, "F")
            time.sleep(0.1)
        
        # 주먹여부처리
        fingers_left = detector.fingersUpDown(left_hand, "Left")
        current_left_state = check_hand_state(fingers_left)
        
        # 상태 유지 로직
        if current_left_state == "Back":
            left_state = "Back"
        elif current_left_state == "Front":
            left_state = "Front"
        elif left_state == "Back" and current_left_state != "Front":
            # Back 상태를 유지
            pass
        else:
            left_state = current_left_state
        
        #아두이노 값 전송
        if left_state == "Back":
            send_left(left_length, "B")
            time.sleep(0.1)
        else:
            send_left(left_length, "F")
            time.sleep(0.1)
        

    # 오른손 처리
    if right_hand:
        # 엄지와 검지 랜드마크 (예: 4번과 8번)
        x1, y1 = right_hand[4][1], right_hand[4][2]
        x2, y2 = right_hand[8][1], right_hand[8][2]
        cx, cy = (x1+x2)//2, (y1+y2)//2
        
        cv2.circle(img, (x1, y1), 5, (0, 0, 255), cv2.FILLED)  # 빨간색
        cv2.circle(img, (x2, y2), 5, (0, 0, 255), cv2.FILLED)  # 빨간색
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)  # 빨간색 선
        
        cv2.circle(img, (cx,cy), 5, (255,0,255), cv2.FILLED)
        
        right_length = int(math.hypot(x2-x1, y2-y1))
        
        if right_length<=15:
            cv2.circle(img, (cx,cy), 8, (0,255,0), cv2.FILLED)
            send_right(0, "F")
            time.sleep(0.1)
        
         #주먹여부처리
        fingers_right = detector.fingersUpDown(right_hand, "Right")
        current_right_state = check_hand_state(fingers_right)
        
        # 상태 유지 로직
        if current_right_state == "Back":
            right_state = "Back"
        elif current_right_state == "Front":
            right_state = "Front"
        elif right_state == "Back" and current_right_state != "Front":
            # Back 상태를 유지
            pass
        else:
            right_state = current_right_state
        
        if right_state == "Back":
            send_right(right_length, "B")
            time.sleep(0.1)
        else:
            send_right(right_length, "F")
            time.sleep(0.1)
    
    # 양손이 주먹인지 확인
    if left_state == "Fist" and right_state == "Fist":
        send_servo("ON")
        send_left(0, "F")
        send_right(0, "F")
    else:
        send_servo("OFF")
        
        
    # display_hand_state 함수 호출
    display_hand_state(img, left_state, right_state, left_hand, right_hand)
    
    # fps 확인
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    
    cv2.putText(img, f'FPS: {int(fps)}', (40, 80), cv2.FONT_HERSHEY_COMPLEX,
                1, (255, 0, 0), 3)
    
    cv2.imshow("Img", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # 'q' 키를 누르면 종료
        break
        
ser.close()
cap.release()
cv2.destroyAllWindows()